## 대화 요약 코드

In [20]:
# 원시 말뭉치 에서 기준에 맞게 말뭉치 추출
# 기준 : 220어절 이상
import json
import glob

json_list = glob.glob("../../말뭉치/원시_말뭉치/대화 요약/json/*.json")
form_list = []
count = 0
for each_json in json_list :
    with open(f"{each_json}") as json_f :
        json_data = json.load(json_f)
        for json_document in json_data["document"] :
            form = ""
            if "[사설]" in json_document["paragraph"][0]["form"] :
                for json_document_paragraph in json_document["paragraph"] :
                    form += json_document_paragraph["form"] + "\n"
                if len(form.split(" ")) >= 220 :
                    switch_json = {}
                    switch_json["id"] = json_document["id"]
                    switch_json["metadata"] = json_document["metadata"]
                    switch_json["paragraph"] = json_document["paragraph"]
                    with open(f"../../말뭉치/정제_말뭉치/대화 요약/json/{json_document['id']}.json", "w") as json_file :
                        json.dump(switch_json, json_file, ensure_ascii=False, indent=4)
                        json_file.close()
                    with open(f"../../말뭉치/정제_말뭉치/대화 요약/txt/{json_document['id']}.txt", "w") as txt_file :
                        txt_file.write(form)
                        txt_file.close()

print(len(form_list))
print(count)


0
0


In [12]:
# 1차 라벨링 결과물을 가지고 핵심 문장을 txt에 추가
import jsonlines
from glob import glob
SOURCE_FILE = "./"

# 각 jsonl loop
with jsonlines.open("../../말뭉치/라벨링_결과물/20240229/admin.jsonl") as jsonl :

    # jsonl 에서 각 문서를 loop
    for line in jsonl :
        sentence_list = []
        print(line)
        print(line['text'])
        for line_entities in line["entities"] :
            sentence_list.append(line['text'][line_entities['start_offset']:line_entities['end_offset']])
        with open("../../말뭉치/라벨링_결과물/20240229/admin_txt.txt", "w") as txt_file :
            txt_file.write(line['text'] + "\n" + "\n" + "[핵심문장]" + "\n"  + "\n".join(sentence_list))
            txt_file.close()
        

{'id': 197, 'text': '합계출산율이 사상 처음 0.6명대로 추락했다. 28일 통계청이 발표한 ‘출생·사망통계’를 보면 지난해 4분기 합계출산율은 0.65명을 기록했다. 2023년 합계출산율도 2022년(0.78명)보다 0.06명 감소한 0.72명까지 내려갔다. 이 추세라면 올해 0.7명선이 무너지게 된다. 정부가 내놓은 수많은 저출생 대책이 추세도 못 바꾸고 효과를 내지 못하는지 냉철히 짚어봐야 한다.\n\n국내 저출생 문제는 전 세계적으로 유례를 찾을 수 없다. 2021년 기준 경제협력개발기구(OECD) 28개국의 합계출산율 평균치(1.58명)의 절반에도 못 미친다. 합계출산율이 1명이 채 안 되는 나라도 한국뿐이다. 정부가 저출생 예산과 대책을 쏟아내지만, 오히려 출산 기피는 더 심화하고 있다. 저출생 극복이 길을 잃었고, 그야말로 백약이 무효다.\n\n전미경제연구소(NBER) 보고서는 출산율이 높은 선진국엔 4가지 특징이 있다고 분석한다. 남성의 적극적인 가사·육아 노동 참여, 워킹맘에 우호적인 사회적 분위기, 정부의 적극적인 가족 정책, 육아를 마친 남녀의 취업 문턱이 낮은 유연한 노동시장 등이다. 한국도 청년 세대의 고용불안, 주거비 부담, 여성에게 집중된 돌봄 부담 등이 저출생 원인이라는 건 이미 알고 있다. 그런데도 출생률 추락 기조가 바뀌지 않는 건 보다 특단의 해법이 있어야 한다는 엄중한 경고일 수 있다.\n\n저출생 극복을 위해서는 경제적 현금 지원 정책을 넘는 국가적인 대전환이 필요하다. 당장 비혼 출산 장벽을 허무는 게 반전의 실마리가 될 수 있다. 결혼 여부와 상관없이 임신·출산·복지 혜택을 주는 프랑스는 2018년 기준 비혼 출산 비율이 60%를 차지했다. 여전히 비혼 출산이 복지 사각지대에 있는 한국에 시사하는 바가 크다. 저출산·고령사회위원장인 대통령이 나서 비혼 출산 가정에도 기혼 가정과 같은 법적 지위를 부여하고 지원 확대를 검토해야 한다. 출산 의지를 꺾는 일자리·성 차별 관행도 바로잡아야 한다. ‘국가 소멸’ 